# Session 3: Multiple comparision correction and report results

In this session, we will learn how to conduct statistical testing of the second-level analysis. It will involve multiple comparision correction, as well as how to report results in a scientifically rigorous way.

## Tools We’ll Use

### **Nilearn**
Nilearn 

## Data for this session
We will use the group-level z-stat map `group_contrast_house_bottle.nii.gz` produced from previous session.

Please run the cell below for the runtime of each notebook. It will import necessary packages that you require to go through this tutorial.

In [ ]:
# --- Basic Setup (always run this first) ---

# Install dependencies 
%pip install -q gdown
%pip install -q git+https://github.com/Yuan-fang/fMRI-tutorial.git

# Import essential packages
import warnings
from pathlib import Path
from nilearn import image, plotting
from nilearn.image import index_img
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.contrasts import compute_fixed_effects
from nilearn.glm.second_level import SecondLevelModel
from nilearn.plotting import plot_design_matrix
from nilearn.image import threshold_img
from nilearn.reporting import get_clusters_table
from bids import BIDSLayout
import module
import pandas as pd
import matplotlib.pyplot as plt

from tutorial.utils.paths import PathManager

warnings.filterwarnings("ignore")

Let's setup the directories (folders).

In [ ]:
# --- Set up data directories ---

DATASET = "Haxby2001" # name of the dataset
BASE_DIR = Path('/neurodesktop-storage/fmri_tutorial')   # base directory for the tutorial
DATA_DIR = BASE_DIR /  DATASET # data directory for the dataset
DERIV_DIR = BASE_DIR / "derivatives" # derivatives directory for processed data

for p in (DATA_DIR, DERIV_DIR):
    p.mkdir(parents=True, exist_ok=True)

# Print out the data directories
print("Data directory:       ", DATA_DIR.resolve())
print("Derivatives directory:", DERIV_DIR.resolve())

As next we will only process the group-level data and not need to access individual subject's data files, we will not use `BIDSLayout` and `PathManager` (their job is to facilicate individual subject's data access). We just need to specify the directory for the group-level data files.

In [ ]:
# specify the group-level directory
grp_dir = DERIV_DIR / "fsl_preproc" / "group_level"

# specify the group-level z-stat map file
# here the name of the z-stat map is "group_contrast_house_bottle.nii.gz" as produced in previous session.
grp_zmap_file = grp_dir / "group_contrast_house_bottle.nii.gz"

# print the group-level z-stat map file path
print("Group-level z-stat map file:", grp_zmap_file.resolve())

Now let's read the zstat map and display it. Without thresholding the image.

In [ ]:
# load the group-level z-stat map
group_zmap = image.load_img(grp_zmap_file)

# plot the group-level z-stat map
# without thresholding
plotting.plot_stat_map(
    group_zmap,
    threshold=None,
    title='Group-level map: House vs Bottle (without thresholding)'
)

You can see, when not threshold the z-stat map, it's a colorful image. Red color denotes positive z-scores (house > bottle), blue denotes negative z-scores (house < bottle).

#### 🤔 Do it yourself: 
Thresholding the z-stat map with more and more stringent thresholds, and see how the thresholding impact the looks of the image.

_Type your answer in the cell below._

In [ ]:
# Plot the group-level z-stat map with a threshold. Play with different threshold values. 
# --- YOUR CODE HERE ---


Let's say, we'd like to get those voxels with z-scores above certain threshold, for example, a z-score which corresponds to p = 0.05 at two-tails.

You can use some online calculators to get the z-score at p=0.05, for example, here:
[p-value to z-score calculator](https://www.gigacalculator.com/calculators/p-value-to-z-score-calculator.php)

In [ ]:
# plot the group-level z-stat map with a threshold corresponding to p=0.05 (two-tailed)
threshold_value = 1.96  # z-score corresponding to p=0.05 (two-tailed)
plotting.plot_stat_map(
    group_zmap,
    threshold=threshold_value,
    title=f'Group-level map: House vs Bottle (thresholded at z={threshold_value})'
)

You can see that some positive blobs survived the above threshold. But how can you report it?

By reporting it, I mean you need to list all those blobs, their coordinates, their spatial extent (voxles), the peak z-stat in each blob.

You also need to show a better image of the above z-stat map, for example, to display it at a specific coordinates.

In [ ]:
# create a report of the thresholded z-stat map
report = get_clusters_table(
    group_zmap,
    stat_threshold=threshold_value
)

# print the report
print(report)

Oh no! We got so many clusters! That's way too many. You can understand that many small clusters with just one voxel (our voxel size is about 2 mm, so corresponds to cluster size of 8 mm3) are meaningless, due to the multiple comparisions. They are more likely just false positives.

So, without formally correcting multiple comparisions, we may set an _arbitary_ cluster size threshold, to filter out those clusters with low size. _This is not a rigorous practice_, but sometimes, researchers have to report their results in this way, when they realized that when formally conducting multiple comparision correction, no cluster can survive.

A less rigorous report is better than nothing to report :) Only thing to keep in mind is that, when you report results in this way, you **MUST** state in your paper the cluster is "uncorrected" (not surving through multiple comparision corretion).

In [ ]:
# Create a report of the thresholded z-stat map with a cluster size threshold
cluster_size_threshold = 30  # in voxels
report = get_clusters_table(
    group_zmap,
    stat_threshold=threshold_value,
    cluster_threshold=cluster_size_threshold
)

# print the report
print(report)

So we got far lesser clusters. We can make the thresholding further stringent. For example, we can set a threshold corresponding to p = 0.001 (two-tailed).

#### 🤔 Do it yourself: 
Got the report at a more stringent threshold at p = 0.001 (two-tailed), with cluster size larger than 30 voxles

_Type your answer in the cell below._

In [ ]:
# --- YOUR CODE HERE ---



From the statistical report, only one large cluster survived thresholding at MNI coordinates (-24, -80, -16). It has a volume of 528 mm3 (roughly 66 voxels at 2x2x2 mm).

Although Figures 1a and 1b show two additional sub-peaks within this cluster, these are secondary local maxima automatically listed by Nilearn.

>In most papers, you do not need to report these subsidiary peaks.

---

#### Interpretation of the Surviving Cluster

To better understand what brain region this coordinate corresponds to, and what its functions might be, you can use a few helpful online tools.

##### 1. Brodmann Area & Talairach Conversion

Use this tool to convert the MNI coordinate to Talairach space and obtain its Brodmann area:

**MNI ↔︎ Talairach Converter (BioImage Suite):** https://bioimagesuiteweb.github.io/webapp/mni2tal.html

This will give you anatomical labels and BA information tied to your cluster.

##### 2. Prior Literature & Meta-analytic Context

To find what prior studies have reported activation near this coordinate, use:

**Neurosynth: Location-based search:**
https://neurosynth.org/locations/

This helps contextualize the cluster within broader neuroimaging literature.

---

#### 🤔 Do it yourself: 
Paste the above MNI to the two websites and explore:
- this MNI coordinates' Talarich coordinates
- its Brodmann label
- its associated meta-analytic coativation map
- its associated functional connectivity map
- its associated cognitive terms
- studies reporting similar coordinates

In your report, you can write something in your _results_ section like:

>"A cluster in the left occipital cortex survived the voxelwise threshold of p < 0.001(uncorrected), with peak activation at MNI coordinates (–24, –80, –16) and an extent of 528 mm³ (peak z-value: 5.18)."

You may also want to produce a thresholded image with this location, and put the produced image into your report.

In [ ]:
# plot the group-level z-stat map with a threshold corresponding to p=0.001 (two-tailed)
plotting.plot_stat_map(
    group_zmap,
    threshold=3.29, # z-score corresponding to p=0.001 (two-tailed)
    cut_coords=(-24, -80, -16),
    title='Group-level map: House vs Bottle (thresholded at z=3.29)',
    draw_cross=False    # disables crosshairs
)

You may find that there are other clusters on your image, becasue the `plotting.plot_stat_map()` can only threshold based on stat, not cluster size. 

If you want to make a cleaner view, only showing the clusters surived the z-stat threshold _as well as_ cluster size, you will need some workaround.

In [ ]:
# Threshold with cluster size filtering
# this actually produce a new image only showing the clusters survived both stat threshold and cluster size threshold
thresholded_map = threshold_img(
    img=group_zmap,
    threshold=3.29,  # z-score corresponding to p=0.001 (two-tailed)
    cluster_threshold=30,  # minimum cluster size in voxels
)

# plot the thresholded map
# because this is a thresholded map already, we do not need to set threshold again in plotting.plot_stat_map()
plotting.plot_stat_map(
    thresholded_map,
    cut_coords=(-24, -80, -16),
    title='Group-level map: House vs Bottle (thresholded at z=3.29)',
    draw_cross=False    # disables crosshairs
)

So far so good, but wait, the above result is uncorrected! To get more rigorous results, we need to conduct multiple comparision correction.

### Multiple comparision correction

As discussed in the lecutures, there are mutliple ways to do multiple comparision correction. In Nilearn, correction is down with the tool `threshold_stats_img()`. Here let's go over them one by one.

##### Bonferroni correction

As you know, this is the most stringent one.

In [ ]:
from nilearn.glm import threshold_stats_img

# Bonferroni correction
# `alpha` here is the family-wise error rate (i.e., false positive rate).
# height_control='bonferroni' specifies the method
# `thresholded_map` is the resulting thresholded image
# `actual_threshold` is the actually used threshold
thresholded_map, actual_threshold = threshold_stats_img(
    stat_img=group_zmap,
    alpha=0.05,
    height_control='bonferroni'
)

print(f'Bonferroni corrected threshold: {actual_threshold}')

As you can see, for Bonferroni correction, it actually uses a super stringent threshold.

We've known that the peak z-value is 5.18, so with the threshold of 5.21, no cluster can surive it. You can verify this using `get_clusters_table` by yourself. 

---

#### FDR correction

FDR (false discovery rate) correction is a more lenient correction method.

FDR controls the expected proportion of false discoveries _among the voxels you declare significant_.


In [ ]:
# FDR correction
# `alpha` here is the expected proportion of false positives among suprathreshold voxels.
# i.e, q-value<0.05
# height_control='fdr' specifies the method
# `thresholded_map` is the resulting thresholded image
# `actual_threshold` is the actually used threshold
thresholded_map, actual_threshold = threshold_stats_img(
    stat_img=group_zmap,
    alpha=0.05,
    height_control='fdr'
)

print(f'FDR corrected threshold: {actual_threshold}')

You can see the actual threshold is lower. Let's see how many clusters survived.

In [ ]:
# get the report of the thresholded map after FDR correction
report = get_clusters_table(
    thresholded_map,
    stat_threshold=0 # because the thresholded_map is already thresholded, we set stat_threshold=0 to get all surviving clusters
)
print(report)

Two clusters survied. However, each cluster just has 1 voxels (2x2x2 mm3)! (Note the group z-stat map is only from 4 subjects, so power is expectedly very low)

You can write something like below in your report: 
>"Voxelwise FDR correction at q < 0.05 revealed two small clusters of significant activation.The first cluster (8 mm³; 1 voxel) showed a peak at MNI [–6, –52, 4], with a peak z-value of 5.19. The second cluster (8 mm³; 1 voxel) showed a peak at MNI [–24, –80, –16], with a peak z-value of 5.18."

---

#### Cluster correction

Unfortunately, Nilearn cannot conduct traditional cluster correction. To carry out cluster cluster, we will need FSL.

In [ ]:
# load FSL (version 6.0.7.8)
# Note other versions may not work with this tutorial
await module.load('fsl/6.0.7.8')
await module.list()

In [ ]:
# Using fslmaths to create a group mask from the group z-stat map
# fslmaths first takes the absolute value of the z-stat map (-abs)
# then binarizes it to create a mask (-bin)

# to-be-created group mask file
grp_mask_file = grp_dir / "group_mask.nii.gz"

# create the group mask
!fslmaths "{grp_zmap_file}" -abs -bin "{grp_mask_file}"

# print the group mask file path
print("Group mask file:", grp_mask_file.resolve())

We can visualize the group mask file. You can see it has all 1s inside the brain, and 0s outside it. Its function is to define an area that an operation will be coducted in.

In [ ]:
# visualize the group mask
plotting.plot_epi(grp_mask_file, title="Group Mask")
plotting.show()

To conduct cluster correction, we need to first estimate the smoothness of our group z-stat map.

In [ ]:
# estimate the smoothness of the group z-stat map within the group mask
!smoothest --mask="{grp_mask_file}" --zstat="{grp_zmap_file}"

The important ones are:
- DLH: the smoothness estimate
- VOLUME: number of voxels in the mask

Let's note down these two parameters.

In [ ]:
# conduct cluster correction using fsl-cluster
# using the parameters obtained from `smoothest`
# --in: group z-stat map
# --thresh: cluster-forming threshold; 3.1 corresponds to p=0.001 (two-tailed)
# --dlh: estimated smoothness (FWHM)
# --volume: total volume of the mask
# --pthresh: cluster-level threshold
# --mm: report in mm

!fsl-cluster --in="{grp_zmap_file}" --thresh=3.1 --dlh=0.0377951 --volume=267676 --pthresh=0.05 --mm

Nice! As you can see, cluster correction reveals 1 significant (p = 0.049) cluster with 154 voxels and peak at MNI coordinates (-24, -80, -16)

You can write in your report in something like:
>"Using Gaussian random field cluster-extent correction (cluster-forming threshold Z > 3.1, cluster-wise FWE p < 0.05), we identified one significant cluster (154 voxels; cluster-level p = 0.0489) with a peak at MNI [−24, −80, −16] (Z = 5.18)."

You may also want to export a thresholded z-map containing only clusters surviving cluster FWE (for example, if you want to make a figure based on this map). To do this, you just need to add an additional argument (--othresh) to `fsl-cluster`

In [ ]:
# to-be-created thresholded z-stat map file after cluster correction
cluster_corrected_zmap_file = grp_dir / "group_contrast_house_bottle_cluster_corrected.nii.gz"

# conduct cluster correction using fsl-cluster and output the thresholded z-stat map
!fsl-cluster --in="{grp_zmap_file}" --thresh=3.1 --dlh=0.0377951 --volume=267676 --pthresh=0.05 --mm --othresh="{cluster_corrected_zmap_file}"